In [1]:
# импортируем необходимые инструменты 
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

In [2]:
# метод для считывания файла и преобразование в список 
def ReadFile(path):

    f = open(path)
    raw = []
    for line in f:    
        raw.append(line)
    f.close()
    return raw



In [3]:
# определяем класс Байесовского классификатора для определения спама
class BayesClassifier():
    
    # конструктор класса
    def __init__(self):
        self.wordDict = {}

    # метод для преобразования строк в массивы
    def TokenizeText(self, raw):
        # импортируем инструмент для преобразования в массивы с удалением знаков пунктуации
        from nltk.tokenize import RegexpTokenizer
        tokenizer = RegexpTokenizer(r'\w+')

        data = []
        for r in raw:
            words = (np.array(tokenizer.tokenize(r))) # токенизируем предложения
            words = [w.lower() for w in words] # приводим слова к нижнему регистру
            data.append(words)

        return np.array(data)

    # метод для разделения данных на массивы признаков (слова в предложении) и ответы (спам или нет) 
    def SplitData(self, data):
        # создаем списки для спам- и хам-предложений
        spam_data = []
        ham_data = []

        for s in data:
            if s[0]=='ham':
                ham_data.append(s[1:]) # если сообщение не является спамом
            else:
                spam_data.append(s[1:]) # если сообщение является спамом


        return ham_data, spam_data

    # метод для удаления стоп-слов и отдельных букв
    def RemoveJunk(self, text):
        # устанавливаем необходимый инструментарий для удаления стоп-слов
        import nltk
        nltk.download('stopwords')

        from nltk.corpus import stopwords
        
        # импортируем английские стоп-слова и добавляем к ним отдельные буквы ввиду неинформативности
        stop_words = set(stopwords.words('english'))
        stop_words = stop_words.union(('q','w','e','r','t','y','u','i','o','p','a','s','d',
                                       'f','g','h','j','k','l','z','x','c','v','b','n','m'))
        
        # удаляем из каждого предложения стоп-слова
        for i in range(len(text)):
            text[i] = [w for w in text[i] if (not w in stop_words) and w.isalpha()]

        return text

    # метод для подготовки данных из файла
    def PrepareData(self, raw):
        data = self.TokenizeText(raw)
        ham_data, spam_data = self.SplitData(data)
        ham_data, spam_data = self.RemoveJunk(ham_data), self.RemoveJunk(spam_data)
        return np.array(ham_data), np.array(spam_data)

    # метод для формирования словаря "спамовости" слов
    def CreateDict(self, ham_data, spam_data): 
        wordDict = {}
        
        # создаем соответствующие ключи для каждого слова 
        for s in spam_data:
            for w in s:
                wordDict[w]=0

        for s in ham_data:
            for w in s:
                wordDict[w]=0
        
        # считаем количество спам- и хам-предложений
        spam_count = len(spam_data)
        ham_count = len(ham_data)   
        
        # для каждого слова рассчитываем "спамовость"
        for w in wordDict:  
            
            # считаем частоту появления в спам- и хам- предложениях
            spam_freq = 0
            ham_freq = 0
            
            for s in spam_data:
                spam_freq += (w in s)

            for s in ham_data:
                ham_freq += (w in s)
            
            # рассчитываем "спамовость" в соответствии с формулой Байеса
            res_prob = spam_freq/(spam_freq+ham_freq)
            
            # заменяем крайние случаи (0/1) на близкие значения
            if res_prob == 1:
                res_prob = 0.99
            elif res_prob == 0:
                res_prob = 0.01
            wordDict[w] = res_prob
        
        return wordDict

    # метод для определения спама в массиве предложений
    def IdentifySpam(self, data):
        res_list = []
        
        # фиксируем обученный словарь "спамовости" слов
        wordDict = self.wordDict
        
        # создаем списки со "спамовостью" для всех входящих в предложение слов
        for newMes in data:
            probs = []
            for w in newMes:
                if w in wordDict:
                    probs.append(wordDict[w])
            
            # находим вероятности принадлежности к ham для расчета условной вероятности по Байесу
            probs_rev = [(1-p) for p in probs]

            top = np.prod(probs)
            bottom = np.prod(probs_rev)
            
            # находим вероятность принадлежности сообщения к спаму
            final_prob = top/(top+bottom)
                        
            res_list.append(final_prob)

        return res_list

    # итоговый метод для обучение классификатора
    def fit(self, x_train):
        ham_data, spam_data = self.PrepareData(x_train)
        self.wordDict = self.CreateDict(ham_data, spam_data)        

    # итоговый метод для предсказания ответов для массива сообщений
    def predict(self, x_test):    
        data = self.TokenizeText(x_test)
        data = self.RemoveJunk(data)
        res_list = self.IdentifySpam(data) 
        return np.array(res_list)

In [4]:
from sklearn.model_selection import train_test_split as tts

# создаем экземпляр классификатора
Bayes = BayesClassifier()

data = ReadFile('spam.txt')

#разделяем данные в отношении 9 к 1 (на обучающую и тестовую выборки)
data_train, data_test = tts(data, train_size = 0.9, test_size = 0.1, random_state=5).copy()

X_test = []
y_test = []

#токенизируем текст
tokenizer = RegexpTokenizer(r'\w+')
data_test_token = Bayes.TokenizeText(data_test)

# сохраняем сообщения, удаляя слова ham/spam в начале и оставляя их в формате строк
for s in (data_test):
    if s[0]=='h':
        X_test.append(s[3:])
    else:
        X_test.append(s[4:])
    
# фиксируем соответствующие ответы для последующей проверки качества
for s in (data_test_token):
    if s[0]=='ham':
        y_test.append(0)
    else:
        y_test.append(1)
        
y_test = np.array(y_test)

In [13]:
# обучаем классификатор
Bayes.fit(data_train)

# предсказываем ответы для отложенной выборки
res = Bayes.predict(X_test)

# сравниваем полученные ответы с изначальными и определяем точность работы классификатора с помощью различных метрик
from sklearn import metrics 

AUC_ROC = metrics.roc_auc_score(y_test, res)
print('\nAUC-ROC: ', str(AUC_ROC))

acc = metrics.accuracy_score(y_test, res>0.5) #значения вероятности больше 0.5 относим к "1", меньше - к "0"
print('Accuracy:', str(acc))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\levin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\levin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\levin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

AUC-ROC:  0.949677531307
Accuracy: 0.985663082437
